In [8]:
from collections import defaultdict, OrderedDict
from itertools import product 

testd = dict(test=[3], bla=[4], blabla=[42, 43, 44])
for i in product(*testd.values()):
    print(i)

(3, 4, 42)
(3, 4, 43)
(3, 4, 44)


In [9]:
testd.values()

dict_values([[3], [4], [42, 43, 44]])

In [11]:
from dataclasses import dataclass
from typing import List

@dataclass
class TestSettings:
    n_comps: List[int]
    mix_ratio: List[float]
    umap_metric: List[str]
    min_cluster_size: List[int]
        
standard_test = TestSettings([3, 15, 45, 200], [0.0, 0.3, 0.15, 0.4], ["cosine"], [15, 45, 90])

standard_test

TestSettings(n_comps=[3, 15, 45, 200], mix_ratio=[0.0, 0.3, 0.15, 0.4], umap_metric=['cosine'], min_cluster_size=[15, 45, 90])

In [28]:
standard_test.__dict__.values()

dict_values([[3, 15, 45, 200], [0.0, 0.3, 0.15, 0.4], ['cosine'], [15, 45, 90]])

In [30]:
for i in product(*standard_test.__dict__.values()):
    print(i)

(3, 0.0, 'cosine', 15)
(3, 0.0, 'cosine', 45)
(3, 0.0, 'cosine', 90)
(3, 0.3, 'cosine', 15)
(3, 0.3, 'cosine', 45)
(3, 0.3, 'cosine', 90)
(3, 0.15, 'cosine', 15)
(3, 0.15, 'cosine', 45)
(3, 0.15, 'cosine', 90)
(3, 0.4, 'cosine', 15)
(3, 0.4, 'cosine', 45)
(3, 0.4, 'cosine', 90)
(15, 0.0, 'cosine', 15)
(15, 0.0, 'cosine', 45)
(15, 0.0, 'cosine', 90)
(15, 0.3, 'cosine', 15)
(15, 0.3, 'cosine', 45)
(15, 0.3, 'cosine', 90)
(15, 0.15, 'cosine', 15)
(15, 0.15, 'cosine', 45)
(15, 0.15, 'cosine', 90)
(15, 0.4, 'cosine', 15)
(15, 0.4, 'cosine', 45)
(15, 0.4, 'cosine', 90)
(45, 0.0, 'cosine', 15)
(45, 0.0, 'cosine', 45)
(45, 0.0, 'cosine', 90)
(45, 0.3, 'cosine', 15)
(45, 0.3, 'cosine', 45)
(45, 0.3, 'cosine', 90)
(45, 0.15, 'cosine', 15)
(45, 0.15, 'cosine', 45)
(45, 0.15, 'cosine', 90)
(45, 0.4, 'cosine', 15)
(45, 0.4, 'cosine', 45)
(45, 0.4, 'cosine', 90)
(200, 0.0, 'cosine', 15)
(200, 0.0, 'cosine', 45)
(200, 0.0, 'cosine', 90)
(200, 0.3, 'cosine', 15)
(200, 0.3, 'cosine', 45)
(200, 0.3, 'co

In [39]:
@dataclass
class TestData:
    path: str
    name: str
    fraction: List[float]
    contamination: List[float]
    seed: List[int]
        
imdb_20news_3splits = TestData(
    ["/home/philipp/projects/dad4td/data/processed/20_news_imdb.pkl"], ["imdb_20news"], [0.15], [0.1], [42, 43, 44])

for i in zip(imdb_20news_3splits.__dict__,imdb_20news_3splits.__dict__.values()):
    print(i)

('path', ['/home/philipp/projects/dad4td/data/processed/20_news_imdb.pkl'])
('name', ['imdb_20news'])
('fraction', [0.15])
('contamination', [0.1])
('seed', [42, 43, 44])


In [35]:
for i in product(*imdb_20news_3splits.__dict__.values()):
    print(i)

('/home/philipp/projects/dad4td/data/processed/20_news_imdb.pkl', 'imdb_20news', 0.15, 0.1, 42)
('/home/philipp/projects/dad4td/data/processed/20_news_imdb.pkl', 'imdb_20news', 0.15, 0.1, 43)
('/home/philipp/projects/dad4td/data/processed/20_news_imdb.pkl', 'imdb_20news', 0.15, 0.1, 44)


In [57]:
%%time
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def get_outlier_data(oe_path, n_oe, seed):
    df_oe = pd.read_pickle(oe_path)
    df_oe = df_oe.iloc[np.random.RandomState(
        seed=seed).permutation(len(df_oe))].head(n_oe)
    df_oe["label"], df_oe["outlier_label"], df_oe["scorable"] = 0, -1, 0
    return df_oe


def label_data(df, seed, labeled_data, outlier_classes):
    df = df[["text", "target", "vecs"]]
    df["scorable"] = 1
    # get all 20 news data
    df = df.where(df.target != -1).dropna()
    # set everything except one class to inlier
    print("Create column outlier_label")
    df["outlier_label"] = -1
    df.loc[~df.target.isin(outlier_classes), "outlier_label"] = 1
    # create labels for UMAP and ivis that
    # are 0 and 1 (derived from the just created outlier labels)
    df["label"] = (df["outlier_label"]+1)/2
    # stratified sample and set unlabeled data based on labeled_data variable
    print("Sample labeled data df")
    df_unlabeled = df[['target']].groupby('target', group_keys=False).apply(
        lambda x: x.sample(frac=1-labeled_data, random_state=seed))
    
    print(df_unlabeled.index)
    df.loc[df_unlabeled.index, "label"] = -1
    print("Data before split:\n")
    print(df.groupby(['label', 'outlier_label']).size(
    ).reset_index().rename(columns={0: 'count'}), "\n")
    return df


def prepare_data(df, outliers, inliers, seed, fixed_cont,
                 labeled_data, n_oe, test_size, oe_path, doc2vec_model, **kwargs):
    print("Only use classes that are in inliers or outliers")
    df = df.where(df.target.isin(
        outliers+inliers)).dropna()
    # label data as inliers and outliers (for scoring) and whether
    # they have labels or not (semi-supervised)
    df = label_data(df, seed, labeled_data, outliers)

    if fixed_cont:
        df = sample_data(df, 1.0, fixed_cont, seed)
        print("Data after adjusting for fixed contamination:\n")
        print(df.groupby(['label', 'outlier_label']).size(
        ).reset_index().rename(columns={0: 'count'}), "\n")

    if n_oe:
        df_oe = get_outlier_data(oe_path, n_oe, seed)
        #df_oe["vecs"] = doc2vec_model.vectorize(df_oe["text"])

    # split train test
    df, df_test = train_test_split(df,
                                   test_size=test_size, random_state=seed,
                                   stratify=df["outlier_label"])
    if n_oe:
        df = df.append(df_oe)

    if -1 in df.label.unique() and df.label.value_counts()[-1] != df.shape[0]:
        if df[(df.label == 0) & (df.outlier_label == -1)].shape[0] == 0:
            print("Adding missing sample for labeled outlier")
            df.loc[((df.label == -1) & (df.outlier_label == -1)
                    ).idxmax(), 'label'] = 0

    print("Training data:\n", df.groupby(['label', 'outlier_label']).size(
    ).reset_index().rename(columns={0: 'count'}), "\n\n")
    print("Test data:\n", df_test.groupby(['label', 'outlier_label']).size(
    ).reset_index().rename(columns={0: 'count'}), "\n\n")

    return df, df_test

def sample_data(df, fraction, contamination, seed):
    X_n = int(df[df.outlier_label==1].shape[0] * fraction)
    y_n = int(X_n * contamination)

    df = df.iloc[np.random.RandomState(seed=seed).permutation(len(df))]
    df = df[df["outlier_label"] == 1].head(X_n).append(
        df[df["outlier_label"] == -1].head(y_n))
    df = df.reset_index(drop=True)
    return df

#data_path = "/home/philipp/projects/dad4td/data/processed/20_news_imdb_vec.pkl"
data_path = "/home/philipp/projects/dad4td/data/raw/QS-OCR-Large/rvl_cdip.pkl"

n_classes = [20000]
n_class = n_classes[0]

params = dict(
    seed=42,
    test_size=0.2,
    labeled_data=0.5,
    fixed_cont=0.1,
    n_oe=False,
    use_nn=True,
    inliers=[0, 1, 2, 11],
    outliers=  [3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15]
)

# load data and get the doc2vec vectors for all of the data used
df_full = pd.read_pickle(data_path)

df_full = df_full.groupby('target', group_keys=False).apply(
    lambda df: df.sample(n=max(n_classes), random_state=42))

df_full["vecs"] = df_full.target.map(lambda x: np.array([1, 2, 3]))
df_full.vecs = df_full.vecs.apply(tuple)


# sample only a portion of the data
df_partial = df_full.groupby('target', group_keys=False).apply(
    lambda df: df.sample(n=n_class, random_state=42))

df, df_test = prepare_data(
                df_partial, oe_path=oe_path, doc2vec_model=None, **params)

df.target.value_counts()

Only use classes that are in inliers or outliers
Create column outlier_label
Sample labeled data df
Merge labeled data df with full df


/home/philipp/projects/dad4td/dad4tdenv/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


Label the outliers as -1 in output df
Data before split:

   label  outlier_label  count
0   -1.0             -1  11469
1   -1.0              1   3987 

Data after adjusting for fixed contamination:

   label  outlier_label  count
0   -1.0             -1    398
1   -1.0              1   3987 

Training data:
    label  outlier_label  count
0   -1.0             -1    318
1   -1.0              1   3190 


Test data:
    label  outlier_label  count
0   -1.0             -1     80
1   -1.0              1    797 


CPU times: user 2min 7s, sys: 18.9 s, total: 2min 26s
Wall time: 4min 21s


2     808
1     807
0     795
11    780
8      34
14     33
3      31
9      28
10     26
4      26
13     24
12     24
7      24
6      24
15     23
5      21
Name: target, dtype: int64

In [34]:
Training datadf_test.target.value_counts()

1     211
0     201
2     196
11    192
9      12
8      12
13      9
4       8
15      7
5       7
14      6
12      5
7       5
10      4
3       3
6       2
Name: target, dtype: int64

In [5]:
import pandas as pd

data_path = "/home/philipp/projects/dad4td/data/raw/QS-OCR-Large/rvl_cdip.pkl"
df = pd.read_pickle(data_path)

df

,filename,target,split,text
0,imagesr/r/g/e/rge31d00/503210033+-0034.txt,3,test,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n
1,imagesc/c/e/j/cej80d00/517306722+-6724.txt,3,test,\nZ Like to Have the Zippo Gia)\n~ SS Pack in ...
2,imagesm/m/r/r/mrr36d00/50603620-3621.txt,14,test,CO-INVESTIGATOR\n\nNAME POSITION/TITLE BIRTHDA...
3,imagesg/g/t/u/gtu29c00/2084573574a.txt,2,test,"Original Message\nFrom Wile, Vivian\nMonday, A..."
4,imagesh/h/o/f/hof08d00/2071783492.txt,9,test,04/18/98 09:37 G1 7+338-8886 MCKAY FRIED Boooz...
...,...,...,...,...
39995,imageso/o/u/k/ouk93f00/0013006838.txt,10,val,The Tobacco Institute\nIndustry Support of Bio...
39996,imagesf/f/f/b/ffb52c00/2074103881.txt,11,val,‘a.coweames\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\...
39997,imagesg/g/h/b/ghb11f00/0001251052.txt,15,val,ATION\nINTIAL MINNESOTA TOBACCO LITIG\nCONFIDE...
39998,imagesl/l/c/k/lck71f00/2016003416.txt,9,val,"\nTHE WASHINGTON STAR\nSaturday, August 3, 196..."


In [6]:
%%time
df["text_len"] = df.text.map(lambda x:len(x))

CPU times: user 174 ms, sys: 3.98 ms, total: 178 ms
Wall time: 177 ms


In [24]:
df.where(df.text_len.between(0,300)).dropna().target.value_counts()

8     24783
4     18870
3     15963
12     9733
2      6934
10     5902
5      5288
9      4645
11     4644
1      3295
13     3225
6      1895
15     1658
7      1372
0      1333
14      596
Name: target, dtype: int64

In [34]:
print(df.shape[0])
df.where(df.text_len.between(0,50)).dropna().reset_index(drop=True)

399999


,filename,target,split,text,text_len
0,imagesr/r/g/e/rge31d00/503210033+-0034.txt,3,test,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n,22.0
1,imagesx/x/a/b/xab71f00/1002977593_1002977622.txt,6,test,\n,1.0
2,imagesj/j/t/o/jto61f00/2050283643.txt,8,test,"Suisy? Wals\n\n\n2050283643,",25.0
3,imageso/o/f/x/ofx31e00/85330046.txt,8,test,,0.0
4,imageso/o/g/y/ogy01d00/517234514+-4517.txt,3,test,vise ezers\n\n\n\n,14.0
...,...,...,...,...,...
46185,imagesy/y/t/p/ytp33f00/0013040253.txt,10,val,\ntesak\n\n536101556\n\nerst aros\n,29.0
46186,imagesn/n/l/x/nlx33e00/2044743126.txt,8,val,,0.0
46187,imagesu/u/x/x/uxx79e00/0000182112.txt,13,val,,0.0
46188,imagesx/x/y/l/xyl40f00/0000340757.txt,4,val,\n285520069,10.0


In [34]:
inliers = [0, 1, 2, 11]
outliers = [3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15]
std_split = [(inliers, outliers, inliers+outliers)]
tl = ([4, 5, 6], [1, 2, 3])
print([(inliers, [j for j in outliers if j != i] ,[j for j in outliers if j == i]) for i in outliers])
std_split

[([0, 1, 2, 11], [4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15], [3]), ([0, 1, 2, 11], [3, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15], [4]), ([0, 1, 2, 11], [3, 4, 6, 7, 8, 9, 10, 12, 13, 14, 15], [5]), ([0, 1, 2, 11], [3, 4, 5, 7, 8, 9, 10, 12, 13, 14, 15], [6]), ([0, 1, 2, 11], [3, 4, 5, 6, 8, 9, 10, 12, 13, 14, 15], [7]), ([0, 1, 2, 11], [3, 4, 5, 6, 7, 9, 10, 12, 13, 14, 15], [8]), ([0, 1, 2, 11], [3, 4, 5, 6, 7, 8, 10, 12, 13, 14, 15], [9]), ([0, 1, 2, 11], [3, 4, 5, 6, 7, 8, 9, 12, 13, 14, 15], [10]), ([0, 1, 2, 11], [3, 4, 5, 6, 7, 8, 9, 10, 13, 14, 15], [12]), ([0, 1, 2, 11], [3, 4, 5, 6, 7, 8, 9, 10, 12, 14, 15], [13]), ([0, 1, 2, 11], [3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 15], [14]), ([0, 1, 2, 11], [3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14], [15])]


[([0, 1, 2, 11],
  [3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15],
  [0, 1, 2, 11, 3, 4, 5, 6, 7, 8, 9, 10, 12, 13, 14, 15])]

In [39]:
from itertools import permutations
pairwise_split = [(x[0], x[1], x[0]+x[1]) for x in list(permutations([[x] for x in range(0, 16) if x not in [3, 8]], 2))]
pairwise_split

[([0], [1], [0, 1]),
 ([0], [2], [0, 2]),
 ([0], [4], [0, 4]),
 ([0], [5], [0, 5]),
 ([0], [6], [0, 6]),
 ([0], [7], [0, 7]),
 ([0], [9], [0, 9]),
 ([0], [10], [0, 10]),
 ([0], [11], [0, 11]),
 ([0], [12], [0, 12]),
 ([0], [13], [0, 13]),
 ([0], [14], [0, 14]),
 ([0], [15], [0, 15]),
 ([1], [0], [1, 0]),
 ([1], [2], [1, 2]),
 ([1], [4], [1, 4]),
 ([1], [5], [1, 5]),
 ([1], [6], [1, 6]),
 ([1], [7], [1, 7]),
 ([1], [9], [1, 9]),
 ([1], [10], [1, 10]),
 ([1], [11], [1, 11]),
 ([1], [12], [1, 12]),
 ([1], [13], [1, 13]),
 ([1], [14], [1, 14]),
 ([1], [15], [1, 15]),
 ([2], [0], [2, 0]),
 ([2], [1], [2, 1]),
 ([2], [4], [2, 4]),
 ([2], [5], [2, 5]),
 ([2], [6], [2, 6]),
 ([2], [7], [2, 7]),
 ([2], [9], [2, 9]),
 ([2], [10], [2, 10]),
 ([2], [11], [2, 11]),
 ([2], [12], [2, 12]),
 ([2], [13], [2, 13]),
 ([2], [14], [2, 14]),
 ([2], [15], [2, 15]),
 ([4], [0], [4, 0]),
 ([4], [1], [4, 1]),
 ([4], [2], [4, 2]),
 ([4], [5], [4, 5]),
 ([4], [6], [4, 6]),
 ([4], [7], [4, 7]),
 ([4], [9], [4, 9])

In [6]:
import pandas as pd
data_path = "/home/philipp/projects/dad4td/data/raw/QS-OCR-Large/rvl_cdip.pkl"
df = pd.read_pickle(data_path)
df.groupby(df.target).apply(lambda d: d.sample(n=2, random_state=42))

filename target  split  \
target                                                                          
0      69601              imagesn/n/b/x/nbx61c00/2084058121.txt      0  train   
       23689               imagesb/b/u/z/buz54d00/506535638.txt      0    val   
1      68929              imagesv/v/c/p/vcp63e00/2030053328.txt      1  train   
       24615              imagesw/w/z/f/wzf76c00/2077321606.txt      1    val   
2      69807             imageso/o/t/f/otf12c00/2085114376a.txt      2  train   
       24041             imagesw/w/g/m/wgm27d00/2072356363b.txt      2    val   
3      67490               imagesy/y/e/h/yeh94d00/505888306.txt      3  train   
       24358         imagesr/r/e/m/rem51d00/513222644+-2654.txt      3    val   
4      70678         imageso/o/u/q/ouq10d00/509132490+-2490.txt      4  train   
       24754         imagesc/c/h/b/chb51d00/517174238+-4240.txt      4    val   
5      69916              imagesq/q/t/o/qto15c00/2505430773.txt      5  train   
       24325   imagesp/p/o/c/poc32e00/2501052252_2501052268.txt      5    val   
6      69765    imagesj/j/h/y/jhy49c00/PUBLICATIONS000434-0.txt      6  train   
       274387  imagesr/r/s/b/rsb02a00/2023380986_2023380997.txt      6  train   
7      70038   imagesb/b/j/q/bjq71a00/2057451990_2057451992.txt      7  train   
       24722              imagesz/z/s/q/zsq14c00/2069720904.txt      7    val   
8      70027              imagesb/b/u/x/bux05c00/2081354438.txt      8  train   
       23973              imagesh/h/n/l/hnl43e00/2026375100.txt      8    val   
9      69889              imagese/e/y/z/eyz53c00/2064818797.txt      9  train   
       24086   imagesl/l/b/q/lbq53e00/2022189908_2022189909.txt      9    val   
10     69826              imagest/t/u/k/tuk35e00/2040646106.txt     10  train   
       23943         imagest/t/r/v/trv76d00/ti16550679_0791.txt     10    val   
11     70145              imagesz/z/u/q/zuq50f00/0000229816.txt     11  train   
       24473         imagesa/a/v/o/avo48d00/2063249971_9973.txt     11    val   
12     72012              imagesc/c/u/s/cus21f00/0001459276.txt     12  train   
       23842         imagest/t/t/i/tti80c00/2041447281_7332.txt     12    val   
13     70930         imagesv/v/b/p/vbp33a00/507315793+-5794.txt     13  train   
       24304              imagesb/b/w/f/bwf76e00/2060040111.txt     13    val   
14     69137           imagesm/m/n/b/mnb99c00/40010921-0923.txt     14  train   
       24285                imagesq/q/u/x/qux81e00/01338791.txt     14    val   
15     69787              imagesf/f/r/q/frq82e00/2054906062.txt     15  train   
       24620              imagesh/h/n/u/hnu46e00/2023954591.txt     15    val   

                                                            text  
target                                                            
0      69601   \nPHILIP MORRIS\nU.S.A,\n‘0 PARKAVENUE, NEWYOR...  
       23689   \nRJ.Reynolds Tobacco Company |\nWinston-Salem...  
1      68929   LAB SERVICES CONSISTENCY REPORT\nDATE: 2/28/93...  
       24615   SF,\n* pmesment ZY vice Paesibevr,nzatoy 2\nEe...  
2      69807   Origa! Message—=\nFrom: Kar Dai SMTP ard@mento...  
       24041   Message —\nFrom Mokmann, Crisine\n‘Sent! Trura...  
3      67490   Water Aekivi hy)\n\n|\nwakea Atbivitey\nAT-/4\...  
       24358   Ve Cate wt CEPL use Crows FO , Lieut Evupnine ...  
4      70678   \n18\n\nwith: Yas) x\nCoep> Yelp 72\nbeo'd - Y...  
       24754   Pepe em mun ee 5237 a238 MAILED ON 5\nSACS. F....  
5      69916   27/88/1997 16:83 +agg1a1e5¢090 KOBAL Pace at\n...  
       24325   Health Aspects of ETS : Presentation to\nJourn...  
6      69765   \n\nertagenmeteerenent\n\n\nElectrophysiologic...  
       274387  feast\nSeeruovcs ye" one a\nBeet mena\ntears\n...  
7      70038   oe e NSO ot 00088 A e\n\nTae PAE\nDATE: 06/26/...  
       24722   rsviule\nfas nome 100 REGULAR FTB\nmt CODE-005...  
8      70027                                        2081354438\n  
       23973   LUX EmBourg\